In [1]:
import pandas as pd
import numpy as np

In [2]:
df_jan = pd.read_parquet("./yellow_tripdata_2023-01.parquet")
df_feb = pd.read_parquet("./yellow_tripdata_2023-02.parquet")

In [4]:
len(df_jan.columns)

19

In [8]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

In [13]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)
df_jan['duration']

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [14]:
df_jan['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [16]:
print(df_jan.shape)
df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
print(df_jan.shape)

(3066766, 20)
(3009173, 20)


In [17]:
3009173/3066766

0.9812202822125979

In [18]:
from sklearn.feature_extraction import DictVectorizer

In [29]:
df_jan[['PULocationID', 'DOLocationID']] = df_jan[['PULocationID', 'DOLocationID']].astype(str)

/var/folders/d0/hgbr2sv168l9b4rw_h5f089m0000gr/T/ipykernel_77147/3509400803.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan[['PULocationID', 'DOLocationID']] = df_jan[['PULocationID', 'DOLocationID']].astype(str)


In [30]:
train_dicts = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [31]:
train_dicts

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [32]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

In [33]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [34]:
len(dv.feature_names_)

515

In [36]:
target = 'duration'

y = df_jan[target].values

In [37]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y)

LinearRegression()

In [38]:
prediction = lr.predict(X_train)

In [40]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y, prediction, squared=False)

7.649261932073194

In [41]:
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']) / pd.Timedelta(minutes=1)
df_feb['duration']

0           1.683333
1           0.233333
2           0.233333
3          32.083333
4          13.300000
             ...    
2913950    19.000000
2913951    11.133333
2913952    14.000000
2913953     7.000000
2913954     9.800000
Name: duration, Length: 2913955, dtype: float64

In [48]:
print(df_feb.shape)
df_feb = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]
print(df_feb.shape)

(2913955, 20)
(2855951, 20)


In [49]:
df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype(str)

/var/folders/d0/hgbr2sv168l9b4rw_h5f089m0000gr/T/ipykernel_77147/1071417625.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype(str)


In [50]:
val_dicts = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [51]:
X_val = dv.transform(val_dicts)

In [52]:
target = 'duration'

y_val = df_feb[target].values

In [53]:
prediction = lr.predict(X_val)

In [54]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_val, prediction, squared=False)

7.811818966384802